# Download MEaSUREs Greenland Quarterly Ice Sheet Monthly Mosaics from SAR and Landsat V005

Short Name/Collection Key: Greenland Quarterly Velocity

Collection ID: C2627046644-NSIDC_ECS

Sources:
https://nsidc.org/data/nsidc-0731/versions/5

https://search.earthdata.nasa.gov/search?q=NSIDC-0731+V005

In [1]:
import numpy as np
import itertools
import requests
import os
import toolbox.IcesheetCHANGES as changes
import toolbox.collection as collection

## Set options

First, set collections key for desired data set from NASA EarthData and set the region name.
For the collection ID, set collections_key using the dictionary below, or use the following instructions to add a new collection to the dictionary. 

Example: Getting a collection ID from EarthData Search: 
[MEaSUREs Annual Antarctic Ice Velocity Maps V001](https://search.earthdata.nasa.gov/search/granules/collection-details?p=C2245171699-NSIDC_ECS&pg[0][v]=f&pg[0][gsk]=-start_date&q=NSIDC-0720%20V001&tl=1686700071.247!3!!)

While viewing the collection on EarthData, as in the above link, follow "View More Info" to visit the CMR page for the collection.
Then, look for the collections ID in the URL or as a tag below the title. 

![Locating Collection ID from CMR page](getting_collectionID.png)


### Collection ID Dictionary
#### MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short Name/Collection Key: Greenland Quarterly Velocity

Collection ID: C2627036252-NSIDC_ECS

Sources:

https://nsidc.org/data/nsidc-0727/versions/5

https://search.earthdata.nasa.gov/search?q=NSIDC-0727%20V005
 


#### MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short Name/Collection Key: Greenland Monthly Velocity

Collection ID: C2627046644-NSIDC_ECS

Sources:

https://nsidc.org/data/nsidc-0731/versions/1

https://search.earthdata.nasa.gov/search?q=NSIDC-0731%20V005

In [2]:
# Print collections in the database (collection.py)
collection.print_collections()

Available collections: 

MEaSUREs Annual Antarctic Ice Velocity Maps V001
Short name: MEaSUREs Antarctic Annual Velocity

MEaSUREs Greenland Quarterly Ice Sheet Velocity Mosaics from SAR and Landsat V005
Short name: MEaSUREs Greenland Quarterly Velocity

MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Short name: MEaSUREs Greenland Monthly Velocity

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height, Version 2
Short name: ATL14 Antarctic Elevation

ATLAS/ICESat-2 L3B Gridded Antarctic and Arctic Land Ice Height Change, Version 2
Short name: ATL15 Antarctic Elevation



SET collection key and region name

In [3]:
collection_key = 'MEaSUREs Greenland Monthly Velocity'
region_name = 'Greenland'       # 'Antarctica' or 'Greenland'
data_type = 'velocity'          # 'velocity' or 'elevation'

In [4]:
collection_id = collection.collection(collection_key)
print('Collection ID: ', collection_id)

Collection ID:  C2627046644-NSIDC_ECS


Next, define two directories on your local drive as follows:

| directory | purpose | 
|-----------|---------|
|`project_folder` | This is the path where output data from the changes module will be stored - the data to be used directly for analysis. | 
|`data_folder` | This is the path where ice velocity and elevation data, from their respective sources, will be stored. The data_folder option was created to facilitate data storage on external drives. |

In [5]:
project_folder = '/Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples'
data_folder='/Volumes/Seagate/CHANGES/data_repository/tutorial'

Initialize the AntarcticCHANGES object - this object will contain all pertinent information to initialize the data grids in your region of interest.

In [6]:
GC = changes.GreenlandCHANGES(project_folder, data_folder, collection_key, collection_id, region_name, data_type)
GC.print_attributes()

Region name:		 Greenland
Collection name:	 MEaSUREs Greenland Monthly Ice Sheet Velocity Mosaics from SAR and Landsat, Version 5
Collection short name:	 MEaSUREs Greenland Monthly Velocity
Collection ID: 		 C2627046644-NSIDC_ECS
Data type:		 Velocity
Projection:		 3413
Download path:		 /Volumes/Seagate/CHANGES/data_repository/tutorial/Greenland/Velocity/MEaSUREs Greenland Monthly Velocity/Data
Metadata path:		 /Users/tara/Documents/SJSU/MLML/Projects/CHANGES/Examples/Greenland/Velocity/Metadata


Set verbose output to true or false

In [7]:
verbose = False
verbose_light = True

## Obtain list of available files for download

In [8]:
# this function is from the ICESat2 data page
def cmr_filter_nested_urls(search_results):
    """Select only the desired data files from CMR response."""
    if 'feed' not in search_results or 'entry' not in search_results['feed']:
        return []

    all_urls = []
    for e in search_results['feed']['entry']:
        granule_id = e['producer_granule_id']
        if verbose:
            print(granule_id)
        urls = []
        # TODO can generalize this with an if statement for use with non-nested data, ie. measures_antarctic_annual
        urls.append(granule_id)
        for link in e['links']:
            if 'href' not in link:
                # Exclude links with nothing to download
                continue    # continue jumps to next iteration in the loop
            if 'inherited' in link and link['inherited'] is True:
                # Why are we excluding these links?
                continue
            if 'rel' in link and 'data#' not in link['rel']:
                # Exclude links which are not classified by CMR as "data" or "metadata"
                continue
            if 'title' in link and 'opendap' in link['title'].lower():
                # Exclude OPeNDAP links--they are responsible for many duplicates
                # This is a hack; when the metadata is updated to properly identify
                # non-datapool links, we should be able to do this in a non-hack way
                continue
            urls.append(link['href'])
        all_urls.append(urls)

    return all_urls

In [9]:
# Build and call the CMR API URL
cmr_query_url = 'https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=' + GC.collection_id + '&page_size=2000'
response = requests.get(cmr_query_url)

print('CMR request URL: ', cmr_query_url)

# print error code based on response
if response.status_code != 200:
    print('ERROR: {}'.format(response.status_code))
search_page = response.json()
# If JSON contains an error message, print the message at the key, 'error'
if 'errors' in search_page:
    print(search_page['errors'])
else: 
    #urls = cmr_filter_urls(search_page)
    urls = cmr_filter_nested_urls(search_page)
    print("Successfully obtained {} URLs.".format(len(urls)))

CMR request URL:  https://cmr.earthdata.nasa.gov/search/granules.json?echo_collection_id=C2627046644-NSIDC_ECS&page_size=2000
Successfully obtained 288 URLs.


## Download the data

In [11]:
def download_files(urls):
    bad_folders = []
    bad_files = 0
    downloaded = 0

    print("Processing " + str(len(urls)) + " folders")
    
    #for folder in urls:
    for i in range(len(urls)):
        folder = urls[i]
        folder_name = folder[0]

        if verbose or verbose_light:
            print("Processing folder " + str(i+1) + "/" + str(len(urls)) + ": " + folder_name)

        not_downloaded = []
        not_downloaded.append(folder_name)

        for link in folder[1:]:
            file_name = link.split('/')[-1]
                
            # if the file exists already, skip it
            if os.path.exists(os.path.join(GC.download_path, file_name)):
                if verbose:
                    print('    ' + file_name + ' already exists. Skipping...', end='\n')
                continue
            #if not file_name.endswith('.tif'):
            #    if verbose:
            #        print('    ' + file_name + ' is not a tif. Skipping...', end='\n')
            #    continue

            # only download the tif files
            if file_name.endswith('.tif'):
                #if not os.path.exists(os.path.join(GC.download_path, folder_name)):
                #    os.makedirs(os.path.join(GC.download_path, folder_name))

                if verbose or verbose_light:
                    # print Downloading + filename and return carriage to beginning of line
                    print('    Downloading ' + file_name, end='\n')     

                r = requests.get(link, allow_redirects=True)
                if r.status_code != 200:    # 200 is the standard response for successful HTTP requests
                    print('    ERROR: ' + str(r.status_code) + '\n')
                    print('    Could not download ' + link + '\n')

                    # Add the link to the list of files that were not downloaded
                    not_downloaded.append([link])
                    bad_files += 1
                    continue

                # write content to file
                #open(os.path.join(GC.data_folder, GC.region_name, 'Velocity', 'MEaSUREs', GC.collection_key, 'Data', folder_name, file_name), 'wb').write(r.content)
                open(os.path.join(GC.download_path, file_name), 'wb').write(r.content)
                downloaded += 1

        if (len(not_downloaded)) > 1:
            bad_folders.append(not_downloaded)

    if (len(bad_folders) > 0):
        print('WARNING: ' + str(bad_files) + ' file(s) were not downloaded.')
    else:
        if verbose:
           print(str(downloaded) + ' files downloaded successfully.')

    return bad_folders
    

In [18]:
def download_nested_files(urls):
    bad_folders = []
    bad_files = 0
    downloaded = 0

    print("Processing " + str(len(urls)) + " folders")
    
    #for folder in urls:
    for i in range(len(urls)):
        folder = urls[i]
        folder_name = folder[0]

        if verbose or verbose_light:
            print("Processing folder " + str(i+1) + "/" + str(len(urls)) + ": " + folder_name)

        not_downloaded = []
        not_downloaded.append(folder_name)

        for link in folder[1:]:
            file_name = link.split('/')[-1]
                
            # if the file exists already, skip it
            if os.path.exists(os.path.join(GC.download_path, folder_name, file_name)):
                if verbose:
                    print('    ' + file_name + ' already exists. Skipping...', end='\n')
                continue
            if not file_name.endswith('.tif'):
                if verbose:
                    print('    ' + file_name + ' is not a tif. Skipping...', end='\n')
                continue
            # only download the tif files
            if file_name.endswith('.tif'):
                if not os.path.exists(os.path.join(GC.download_path, folder_name)):
                    os.makedirs(os.path.join(GC.download_path, folder_name))

                if verbose or verbose_light:
                    # print Downloading + filename and return carriage to beginning of line
                    print('    Downloading ' + file_name, end='\n')     

                r = requests.get(link, allow_redirects=True)
                if r.status_code != 200:    # 200 is the standard response for successful HTTP requests
                    print('    ERROR: ' + str(r.status_code) + '\n')
                    print('    Could not download ' + link + '\n')

                    # Add the link to the list of files that were not downloaded
                    not_downloaded.append([link])
                    bad_files += 1
                    continue

                # write content to file
                #open(os.path.join(GC.data_folder, GC.region_name, 'Velocity', 'MEaSUREs', GC.collection_key, 'Data', folder_name, file_name), 'wb').write(r.content)
                open(os.path.join(GC.download_path, folder_name, file_name), 'wb').write(r.content)
                downloaded += 1

        if (len(not_downloaded)) > 1:
            bad_folders.append(not_downloaded)

    if (len(bad_folders) > 0):
        print('WARNING: ' + str(bad_files) + ' file(s) were not downloaded.')
    else:
        if verbose:
           print(str(downloaded) + ' files downloaded successfully.')

    return bad_folders
    

In [ ]:
def run_download_files(urls, nested):
    # Download the files
    if nested:
        not_downloaded = download_nested_files(urls)
    else:
        not_downloaded = download_files(urls)

    # Attempt to download any files that were not downloaded the first time
    if len(not_downloaded) > 0:       
        print('Attempting to download these files once more...')
        if nested:
            not_downloaded = download_nested_files(not_downloaded)
        else: 
            not_downloaded = download_files(not_downloaded)
        if len(not_downloaded) > 0:
            print('WARNING: some files were not downloaded.')
            print('Please download the files manually and place in the corresponding folder:')
            print('See file: ' + os.path.join(GC.download_path, 'not_downloaded.txt'))
            print('\n')
            
            # store the list of files that were not downloaded in a text file
            with open(os.path.join(GC.download_path, 'not_downloaded.txt'), 'w') as f:
                for item in not_downloaded:
                    f.write("%s\n" % item)
    if len(not_downloaded) == 0:
        print('All files downloaded successfully.')
    return not_downloaded

In [13]:
not_downloaded = run_download_files(urls, nested=False)

Processing 288 folders
Processing folder 1/288: GL_vel_mosaic_Monthly_01Dec14_31Dec14_SAR_shape_v05.0
Processing folder 2/288: GL_vel_mosaic_Monthly_01Dec14_31Dec14_LS_shape_v05.0
Processing folder 3/288: GL_vel_mosaic_Monthly_01Dec14_31Dec14_velocity_v05.0
Processing folder 4/288: GL_vel_mosaic_Monthly_01Jan15_31Jan15_SAR_shape_v05.0
Processing folder 5/288: GL_vel_mosaic_Monthly_01Jan15_31Jan15_velocity_v05.0
Processing folder 6/288: GL_vel_mosaic_Monthly_01Jan15_31Jan15_LS_shape_v05.0
Processing folder 7/288: GL_vel_mosaic_Monthly_01Feb15_28Feb15_SAR_shape_v05.0
Processing folder 8/288: GL_vel_mosaic_Monthly_01Feb15_28Feb15_LS_shape_v05.0
Processing folder 9/288: GL_vel_mosaic_Monthly_01Feb15_28Feb15_velocity_v05.0
Processing folder 10/288: GL_vel_mosaic_Monthly_01Mar15_31Mar15_velocity_v05.0
Processing folder 11/288: GL_vel_mosaic_Monthly_01Mar15_31Mar15_LS_shape_v05.0
Processing folder 12/288: GL_vel_mosaic_Monthly_01Mar15_31Mar15_SAR_shape_v05.0
Processing folder 13/288: GL_vel_m